In [47]:
import numpy as np
import random

from utils.gradcheck import gradcheck_numeric
from utils.utils import normalizeRows, softmax

import torch

from word2vec import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


What are we going to implement in this assignment? Well we need to implement (both for `naive softmax` and `negative sampling`):

- loss function;
- backward pass;
- SGD;

Let's start with `naive softmax`.

## Naive Softmax

So we have to implement `naiveSoftmaxLossAndGradient(centerWordVec, outsideWordIdx, outsideVectors, dataset)`. The main problem here is to understand what all of these arguments mean. Fortunately we have test examples in 2019 version of the course (albeit in not very friendly format) and so we can debug.

In [48]:
# setup for training (we don't need dataset for naive softmax)
dataset = type('dummy', (), {})()

def dummySampleTokenIdx():
    return random.randint(0, 4)

def getRandomContext(C):
    tokens = ["a", "b", "c", "d", "e"]
    return tokens[random.randint(0, 4)], \
           [tokens[random.randint(0, 4)] for i in range(2 * C)]

dataset.sampleTokenIdx = dummySampleTokenIdx
dataset.getRandomContext = getRandomContext

random.seed(31415)
np.random.seed(9265)
dummy_vectors = normalizeRows(np.random.randn(10, 3))
dummy_tokens = dict([("a", 0), ("b", 1), ("c", 2), ("d", 3), ("e", 4)])

In [49]:
dummy_vectors

array([[-0.96735714, -0.02182641,  0.25247529],
       [ 0.73663029, -0.48088687, -0.47552459],
       [-0.27323645,  0.12538062,  0.95374082],
       [-0.56713774, -0.27178229, -0.77748902],
       [-0.59609459,  0.7795666 ,  0.19221644],
       [-0.6831809 , -0.04200519,  0.72904007],
       [ 0.18289107,  0.76098587, -0.62245591],
       [-0.61517874,  0.5147624 , -0.59713884],
       [-0.33867074, -0.80966534, -0.47931635],
       [-0.52629529, -0.78190408,  0.33412466]])

In [50]:
loss, gradCenterVecs, gradOutsideVectors = \
    skipgram(currentCenterWord="c", 
             windowSize=3, 
             outsideWords=["a", "b", "e", "d", "b", "c"],
             word2Ind=dummy_tokens, 
             centerWordVectors=dummy_vectors[:5, :], 
             outsideVectors=dummy_vectors[5:, :], 
             dataset=dataset)

In [51]:
loss

11.16610900153398

In [70]:
gradCenterVecs.shape

(5, 3)

In [71]:
gradCenterVecs

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [-1.26947339, -1.36873189,  2.45158957],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [79]:
gradOutsideVectors

array([[-0.41045956,  0.18834851,  1.43272264],
       [ 0.38202831, -0.17530219, -1.33348241],
       [ 0.07009355, -0.03216399, -0.24466386],
       [ 0.09472154, -0.04346509, -0.33062865],
       [-0.13638384,  0.06258276,  0.47605228]])

### parameters

From comments:

- `centerWordVec` - numpy ndarray, center word's embedding (`v_c` in the pdf handout)
- `outsideWordIdx` - integer, the index of the outside word (`o` of `u_o` in the pdf handout)
- `outsideVectors` - outside vectors (rows of matrix) for all words in vocab (`U` in the pdf handout)

In [52]:
# currentCenterWord = 'c', index of it in dummy_tokens is 2
# and we make a lookup in the first part of dummy_vectors
centerWordVec = dummy_vectors[2, :]
centerWordVec

array([-0.27323645,  0.12538062,  0.95374082])

In [53]:
# if outsideWord = 'a' then we look up it's index in dummy_tokens
outsideWordIdx = 0

In [54]:
# outsideVectors is the second part of dummy_vectors
dummy_vectors[5:, :]

array([[-0.6831809 , -0.04200519,  0.72904007],
       [ 0.18289107,  0.76098587, -0.62245591],
       [-0.61517874,  0.5147624 , -0.59713884],
       [-0.33867074, -0.80966534, -0.47931635],
       [-0.52629529, -0.78190408,  0.33412466]])

### loss

We first compute probability using formula `(1)` from pdf and then loss using formula `(2)`. But how can we get those probs in code? It seems we may multiply `outsideVectors` and `centerWordVec` to get vector of shape `(5,)`, then use `softmax`. Now we may just take necessary component of it depending of index `o`. As we can see we get correct loss.

In [55]:
currentCenterWord="c"
centerWordVec = dummy_vectors[2, :]
outsideWords=["a", "b", "e", "d", "b", "c"]
outsideVectors=dummy_vectors[5:, :]

In [56]:
outsideVectors.shape, centerWordVec.shape

((5, 3), (3,))

In [57]:
dot_product = centerWordVec.dot(outsideVectors.T)
dot_product.shape

(5,)

In [58]:
outsideVectors[0, :]

array([-0.6831809 , -0.04200519,  0.72904007])

In [59]:
outsideVectors[0, :].dot(centerWordVec)

0.876718551316616

In [60]:
dot_product[0]

0.876718551316616

In [61]:
dot_product_softmax = softmax(dot_product)

In [62]:
dot_product_softmax

array([0.41703564, 0.10030664, 0.12391154, 0.10888915, 0.24985703])

In [63]:
loss_man = 0
probs = softmax(centerWordVec.dot(outsideVectors.T))
for outsideWord in outsideWords:
    outsideWordIdx = dummy_tokens[outsideWord]
    loss_man += -np.log(probs[outsideWordIdx])

In [64]:
loss_man

11.16610900153398

### gradient for center vector

#### theory - 1

So we need to compute gradient of `(3)` with respect to $v_c$. First let's write down loss function and probabilities. The last equation here is an exersice 1 (a).

$$J=CE(y, \hat{y})=-\sum_{w}log(\hat{y}_w)=-log(\hat{y}_o)$$

$$\hat{y}_o=p(o|c)=\frac{\exp(u^T_o v_c)}{\sum_{w}\exp(u^T_w v_c)}$$

Now let's take a partial derivative with respect to a component of $v_c$:  $\partial J /\partial v_{cj}$.

$$\frac{\partial J}{\partial v_{cj}} = \frac{\partial (-log(\hat{y}_o))}{\partial v_{cj}} = \frac{\partial (-u^T_o v_c + \log(\sum_{w}\exp(u^T_w v_c)))}{\partial v_{cj}}$$

First term is just $-u_{oj}$. To compute the second term we need to understand that:

$$\frac{\exp(u^T_{w^\prime} v_c)}{\sum_{w}\exp(u^T_w v_c))}=\hat{y}_{w^\prime}$$

Finally, we get (with or without index `j`):

$$\frac{\partial J}{\partial v_{cj}} = -u_{oj} + \sum_{w} \hat{y}_w u_{wj}$$
$$\frac{\partial J}{\partial v_{c}} = -u_{o} + \sum_{w} \hat{y}_w u_{w}$$

There are two questions left: 

- can we get this result directly without using index `j`? and 
- also can we get vectorized representation of this result;

Let's use the following formula: $\partial (u^T v) / \partial v = u$ that can be checked directly. Now we can get first term directly. And probably the second one as well.

Now consider $U y$ where *columns* of $U$ consists of our $u_i$ vectors. If $y$ is one-hot-encoded with $y_o = 1$ then $U y = u_o$. So the first term in our formula is just $-Uy$. 

The second term is just a combination of columns of $U$ and we know that this is the same as $U \hat{y}$. So we simply get:

$$\frac{\partial J}{\partial v_{c}} = U (\hat{y} - y)$$

This concludes our derivation. Probably next time we won't be so detailed.

#### theory - 2

Next question - how should we accumulate those gradients for `center` vector while we're iterating over the window? If we look at the `skip-gram` loss function (for simplicity we skip all indicies of $w$ - see detailed formula in `pdf`) then pretty much obvious that we have to sum up those gradients.

$$J_{skip-gram}(v_c, w, U) = \sum_{w} J(v_c, w, U)$$

The only thing that we should be careful - we compute here gradient of only one vector $v_c$. So we have to write code like this: `gradCenterVecs[currentCenterWordIdx] += gradCenterVec`.

#### code

Let's now try to get these gradients in our example. `probs` is the same as $\hat y$.

In [68]:
probs.shape

(5,)

In [74]:
centerWordVectors = dummy_vectors[:5, :]
gradCenterVecs = np.zeros(centerWordVectors.shape)
gradCenterVecs.shape

(5, 3)

In [75]:
currentCenterWord="c"
word2Ind = dummy_tokens
currentCenterWordIdx = word2Ind[currentCenterWord]
currentCenterWordIdx

2

In [77]:
for outsideWord in outsideWords:
    # compute grad for one outside word
    outsideWordIdx = word2Ind[outsideWord]
    y = np.zeros(len(word2Ind))  
    y[outsideWordIdx] = 1
    gradCenterVec = outsideVectors.T.dot(probs - y)
    
    # add it to accumulated amount
    gradCenterVecs[currentCenterWordIdx] += gradCenterVec

In [78]:
gradCenterVecs

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [-1.26947339, -1.36873189,  2.45158957],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

### gradient for outside vectors

We are not going to reproduce here detailed derivations and just state the formula.

$$\frac{\partial J}{\partial U} = v_c (\hat{y} - y)^T$$

Now let's try to reproduce computations.

In [83]:
gradOutsideVectors = np.zeros(outsideVectors.shape)

In [84]:
for outsideWord in outsideWords:
    # compute grad for one outside word
    outsideWordIdx = word2Ind[outsideWord]
    y = np.zeros(len(word2Ind))  
    y[outsideWordIdx] = 1
    gradOutsideVecs = np.outer((probs - y), centerWordVec)
    
    # add it to accumulated amount
    gradOutsideVectors += gradOutsideVecs

In [85]:
gradOutsideVectors

array([[-0.41045956,  0.18834851,  1.43272264],
       [ 0.38202831, -0.17530219, -1.33348241],
       [ 0.07009355, -0.03216399, -0.24466386],
       [ 0.09472154, -0.04346509, -0.33062865],
       [-0.13638384,  0.06258276,  0.47605228]])

That concludes our computations for `naive softmax case`. Next we have to look at `negative sampling`.